### Configuration

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import mne

import plotly.express as px
from scipy.stats import zscore
from mne.time_frequency import psd_array_welch
from mne.preprocessing import annotate_amplitude
from derivative import dxdt
from sklearn.neighbors import LocalOutlierFactor
#from pyriemann.clustering import Potato

from utils__helpers_macro import robust_zscore
import utils__config

In [2]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

In [3]:
fif_path = 'Cache/Subject05/Jul11/S05_Jul11_256hz.fif'
out_path = 'Cache/Subject05/Jul11/S05_bad_channels.csv'

In [4]:
sampling_freq = 256
lof_threshold = -2
lof_neighbors = 10

### Load Data

In [5]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = False)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

C:\Users\lal85\AppData\Local\Temp\2\ipykernel_27472\1609515138.py:1: RuntimeWarning: This filename (Cache/Subject05/Jul11/S05_Jul11_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = False)


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<Raw | S05_Jul11_256hz.fif, 61 x 8921600 (34850.0 s), ~4.05 GB, data loaded>

### Manual Review of Welch PSD

In [6]:
ts_data = raw.get_data()
#raw.plot_psd()

# Compute Power Spectral Density (PSD) using Welch's method
psd = psd_array_welch(x = ts_data,
                      sfreq = sampling_freq,
                      fmin = 0.3,
                      fmax = 85,
                      n_jobs = 6)

# Extract data from PSD output
psd_data = psd[0]
freq_labels = pd.Series(psd[1]).astype('int64').to_list()

# Convert to Xarray as an intermediate step in
# getting data into Pandas long (2d) format:
tfr = xr.DataArray(psd_data,
                   dims = ('channel', 'frequency'),
                   coords = {'channel' : raw.ch_names,
                             'frequency' : freq_labels})

tfr = tfr.to_dataframe(name = 'power').reset_index()

# Calculate log-power
tfr['log_power'] = 10 * np.log10(tfr['power'])

Effective window size : 1.000 (s)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   6 | elapsed:    6.9s remaining:    6.9s
[Parallel(n_jobs=6)]: Done   6 out of   6 | elapsed:    8.7s finished


In [7]:
# Dynamic plot with Plotly
fig = px.line(tfr, x = 'frequency', y = 'log_power', color = 'channel', template = 'plotly_white', width = 600, height = 500)
fig.update_layout(xaxis_title = 'Frequency', yaxis_title = 'Log Power', legend_title = 'Channel')
fig.show()

### Flatline Detector

In [8]:
flat_annotations, flat_channels = annotate_amplitude(raw = raw,
                                                     peak = None, # maximum peak-to-peak amplitude above which is bad
                                                     flat = 1e-8, # minimum peak-to-peak amplitude below which is bad
                                                     bad_percent = 95, # percent of time the peak-to-peak minimum is present
                                                     min_duration = 5 # minimum duration of bad segment to be annotated
                                                     )

Finding segments below or above PTP threshold.


In [9]:
flat_annotations.to_data_frame()

,onset,duration,description


In [10]:
flat_channels

[]

### Feature Engineering for LOF

LOF + Flatline Detector validated for scalp EEG: https://doi.org/10.3390/s22197314

Low Frequency (0 - 50 Hz)

In [11]:
# Copy data and bandpass
low_data = raw.copy()
low_data.filter(l_freq = None, h_freq = 50, n_jobs = 6)
low_data = low_data.get_data()

# Convert to within-channel Z-Scores
#low_data = zscore(low_data, axis = 0)

# Run LOF with input (n_samples, n_features)
# (note that fit_predict() will return a boolean array
#  indicating inlier status; but it also modifies the 
#  LOF object and stores the actual LOF scores in an
#  object attribute called negative_outlier_factor_)
lof_low = LocalOutlierFactor(n_neighbors = lof_neighbors)
_ = lof_low.fit_predict(low_data)

# Extract LOF scores, munge, and merge
low_features = lof_low.negative_outlier_factor_

low_features = pd.DataFrame({'channel' : raw.ch_names,
                             'lof_low' : low_features})

#lof_feats = mf_features.merge(low_features, left_on = 'channel', right_on = 'channel')
lof_feats = low_features

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 69 samples (0.270 s)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    1.8s
[Parallel(n_jobs=6)]: Done  61 out of  61 | elapsed:    7.7s finished


High Frequency (50 - 120 Hz)

In [12]:
# See low-frequency section for comments
high_data = raw.copy()
high_data.filter(l_freq = 50, h_freq = 120, n_jobs = 6)
high_data = high_data.get_data()

#high_data = zscore(high_data, axis = 0)

lof_high = LocalOutlierFactor(n_neighbors = lof_neighbors)
_ = lof_high.fit_predict(high_data)

high_features = lof_high.negative_outlier_factor_

high_features = pd.DataFrame({'channel' : raw.ch_names,
                              'lof_high' : high_features})

lof_feats = lof_feats.merge(high_features, left_on = 'channel', right_on = 'channel')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 50 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 50.00
- Lower transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 43.75 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 8.00 Hz (-6 dB cutoff frequency: 124.00 Hz)
- Filter length: 107 samples (0.418 s)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done  61 out of  61 | elapsed:    6.5s finished


First Derivative

In [13]:
# We need to extract both the voltages and the
# time values themselves to calculate derivative
raw_data, raw_times = raw.get_data(return_times = True)

# There are many methods to numerically solve for the derivative
# (versus analytically solving); they can be divided into local and
# global methods, of which the global is the fancier; the simplest
# approximation uses a local method called the Finite Difference.
for channel in range(raw_data.shape[0]):

    ch_dx = dxdt(x = raw_data[channel, :], t = raw_times, kind = 'finite_difference', k = 3)
    
    if channel == 0:
        dx_data = ch_dx[np.newaxis, :]

    else:
        ch_dx = ch_dx[np.newaxis, :]
        dx_data = np.append(dx_data, ch_dx, axis = 0)

# Z-Score data
#dx_data = zscore(dx_data, axis = 0)

# Calculate LOF
lof_dx = LocalOutlierFactor(n_neighbors = lof_neighbors)
_ = lof_dx.fit_predict(dx_data)

dx_features = lof_dx.negative_outlier_factor_

dx_features = pd.DataFrame({'channel' : raw.ch_names,
                            'lof_dx' : dx_features})

# Merge with other features
lof_feats = lof_feats.merge(dx_features, left_on = 'channel', right_on = 'channel')

### Rejection Thresholding

In [14]:
lof_feats.describe()

,lof_low,lof_high,lof_dx
count,61.000000,61.000000,61.000000
mean,-1.467305,-1.456136,-1.416094
std,1.678526,1.502770,1.314315
min,-13.178036,-12.136380,-10.920626
25%,-1.194480,-1.318733,-1.310518
50%,-1.048286,-1.090736,-1.097921
75%,-0.998018,-1.013127,-1.011703
max,-0.977007,-0.965804,-0.971325


In [15]:
# Set the string column to index so that 
# the next selection step can work (it 
# parses all columns and can only take numbers)
lof_feats = lof_feats.set_index(['channel'])

# Create boolean indicating if any of the three LOF
# scores were less than the selection threshold
lof_feats['lof_any'] = 0
lof_feats.loc[(lof_feats < lof_threshold).any(axis = 1), 'lof_any'] = 1

# Keep only channels where LOF < threshold for any of three time series
lof_feats = lof_feats[lof_feats.lof_any == 1]

# Save to CSV and print results
lof_feats.to_csv(out_path, index = True)
lof_feats

,lof_low,lof_high,lof_dx,lof_any
channel,,,,
RPHC4,-5.267572,-4.276835,-3.357245,1
RPC1,-2.349755,-2.274241,-1.729442,1
RSMA5,-1.194480,-1.318733,-2.291408,1
RSMA6,-1.194839,-1.542648,-2.246230,1
RAC3,-2.230013,-1.957066,-1.527578,1
RPHC1,-3.061390,-3.048966,-2.320165,1
RPHC2,-13.178036,-12.136380,-10.920626,1
RPHC3,-3.662304,-3.127684,-2.573747,1


In [16]:
# Manually inspect rejected channels
rejected_channels = lof_feats.reset_index()
raw.pick_channels(ch_names = rejected_channels['channel'].tolist())

#raw.plot()

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<Raw | S05_Jul11_256hz.fif, 8 x 8921600 (34850.0 s), ~544.5 MB, data loaded>